# 🧪 LAB 6 – Workflows + Observabilidad + Visualización

Idea del laboratorio

Definimos un agente de Service Desk con ChatAgent (llamando a tu modelo en GitHub).

Lo envolvemos en un AgentExecutor (ejecutor integrado del framework).

Creamos un TurnManager que:

Recibe el texto del usuario.

Crea un AgentExecutorRequest con un ChatMessage(Role.USER, text=...).

Envía esa request al AgentExecutor.

Recibe un AgentExecutorResponse y saca el texto final como salida de workflow.

Construimos un Workflow con:

TurnManager → AgentExecutor → TurnManager

Activamos Observabilidad (setup_observability) y visualizamos el workflow con WorkflowViz.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

GITHUB_ENDPOINT = os.getenv("GITHUB_ENDPOINT")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_MODEL = os.getenv("GITHUB_MODEL")  # ej: gpt-4.1-mini, gpt-4o, etc.

if not all([GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL]):
    raise RuntimeError(
        "❌ Faltan variables en .env. Debes definir GITHUB_ENDPOINT, GITHUB_TOKEN y GITHUB_MODEL."
    )

# Adaptar a lo que espera OpenAIChatClient
os.environ["OPENAI_API_KEY"] = GITHUB_TOKEN
os.environ["OPENAI_BASE_URL"] = GITHUB_ENDPOINT
os.environ["OPENAI_CHAT_MODEL_ID"] = GITHUB_MODEL

print("✅ Configuración cargada desde .env")
print("  ENDPOINT:", GITHUB_ENDPOINT)
print("  MODEL:", GITHUB_MODEL)


✅ Configuración cargada desde .env
  ENDPOINT: https://models.github.ai/inference
  MODEL: gpt-4o


In [2]:
from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatAgent,
    ChatMessage,
    Executor,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    handler,
)
from agent_framework.openai import OpenAIChatClient
from agent_framework import WorkflowViz
from agent_framework.observability import setup_observability

import asyncio

# Activar OpenTelemetry + logs del framework
setup_observability(enable_sensitive_data=True)
print("✅ Observabilidad habilitada.")


✅ Observabilidad habilitada.


In [3]:
# Creamos el cliente de chat usando las variables OPENAI_*
chat_client = OpenAIChatClient()  # lee OPENAI_API_KEY, OPENAI_BASE_URL, OPENAI_CHAT_MODEL_ID

SERVICE_DESK_INSTRUCTIONS = """
Eres un agente de Service Desk interno.

Tu trabajo es:
- Entender la solicitud del usuario en lenguaje natural.
- Interpretar si es una duda, una petición de vacaciones, un problema técnico, etc.
- Responder de forma clara, profesional y en español.
- Si corresponde, indica qué tipo de ticket abrirías (IT, RRHH, Facilities) y la prioridad sugerida.
- No devuelvas JSON, responde en texto libre.
"""

service_desk_agent = ChatAgent(
    chat_client=chat_client,
    name="ServiceDeskAgent",
    instructions=SERVICE_DESK_INSTRUCTIONS,
)

print("✅ ChatAgent de Service Desk creado.")


✅ ChatAgent de Service Desk creado.


AgentExecutor es un ejecutor integrado del framework que sabe:

- Recibir AgentExecutorRequest.
- Ejecutar el agente (ChatAgent) con esos mensajes.
- Emitir AgentExecutorResponse hacia abajo en el workflow.

In [4]:
agent_executor = AgentExecutor(agent=service_desk_agent, id="service_desk_exec")
print("✅ AgentExecutor creado para ServiceDeskAgent.")


✅ AgentExecutor creado para ServiceDeskAgent.


TurnManager (Executor que coordina la llamada al agente)

In [5]:
class TurnManager(Executor):
    """
    Executor que coordina la llamada al agente de Service Desk mediante AgentExecutorRequest/Response.

    Flujo:
    - Primer handler ('start'): recibe el texto del usuario (str) y lo envía al AgentExecutor
      como AgentExecutorRequest(messages=[ChatMessage(Role.USER, text=...)]).
    - Segundo handler ('on_agent_response'): recibe AgentExecutorResponse y produce la
      salida final del workflow con ctx.yield_output().
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "turn_manager")

    @handler
    async def start(self, user_text: str, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
        """
        Primer paso del workflow: se llama con el texto del usuario.
        Construimos un AgentExecutorRequest con un único ChatMessage de rol USER.
        """
        user_msg = ChatMessage(Role.USER, text=user_text)
        request = AgentExecutorRequest(messages=[user_msg], should_respond=True)
        print("🧭 [TurnManager.start] Enviando AgentExecutorRequest al agente.")
        await ctx.send_message(request)

    @handler
    async def on_agent_response(
        self,
        result: AgentExecutorResponse,
        ctx: WorkflowContext[None, str],
    ) -> None:
        """
        Segundo paso: el AgentExecutor responde con AgentExecutorResponse.
        Aquí extraemos el texto final y lo emitimos como salida del workflow.
        """
        # AgentExecutorResponse tiene un AgentRunResponse dentro (agent_run_response).
        # Extraemos el texto. Si no hubiera .text, extraeríamos del último mensaje.
        run_resp = result.agent_run_response
        text = getattr(run_resp, "text", None)

        if not text:
            # Fallback: concatenar textos de los mensajes de respuesta
            msgs = getattr(run_resp, "messages", [])
            textos = []
            for m in msgs:
                if hasattr(m, "text") and m.text:
                    textos.append(m.text)
            text = "\n".join(textos) if textos else "(sin contenido)"

        print("📨 [TurnManager.on_agent_response] Texto del agente:\n", text)
        await ctx.yield_output(text)


In [6]:
turn_manager = TurnManager(id="turn_manager")

workflow = (
    WorkflowBuilder()
    .set_start_executor(turn_manager)
    .add_edge(turn_manager, agent_executor)    # TurnManager -> AgentExecutor
    .add_edge(agent_executor, turn_manager)    # AgentExecutor -> TurnManager (respuesta)
    .build()
)

print("✅ Workflow construido con TurnManager + AgentExecutor.")


✅ Workflow construido con TurnManager + AgentExecutor.


{
    "name": "workflow.build",
    "context": {
        "trace_id": "0x2ecb1f3182a2064b2afe461384f43f50",
        "span_id": "0x233810dbb5495775",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-11-28T12:34:18.514362Z",
    "end_time": "2025-11-28T12:34:18.514999Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "workflow.id": "b5e2a07f-19f6-4138-9972-c15c1ebdfff9",
        "workflow.definition": "{\"id\": \"b5e2a07f-19f6-4138-9972-c15c1ebdfff9\", \"start_executor_id\": \"turn_manager\", \"max_iterations\": 100, \"edge_groups\": [{\"id\": \"InternalEdgeGroup/95c94540-5fb3-46c3-bb2d-4599710ab87b\", \"type\": \"InternalEdgeGroup\", \"edges\": [{\"source_id\": \"internal:turn_manager\", \"target_id\": \"turn_manager\"}]}, {\"id\": \"InternalEdgeGroup/70b43dcc-b01c-4acb-9790-ee4fd113d39d\", \"type\": \"InternalEdgeGroup\", \"edges\": [{\"source_id\": \"internal:service_desk_exec\", \"targ

In [7]:
async def run_service_desk_workflow(input_text: str):
    print("\n" + "=" * 80)
    print(f"🚀 Ejecutando workflow Service Desk con input:\n\"{input_text}\"\n")

    async for event in workflow.run_stream(input_text):
        print("📡 Evento:", event)
        if isinstance(event, WorkflowOutputEvent):
            print("\n✅ Resultado final del workflow:\n")
            print(event.data)
            print("\n" + "=" * 80)


# Pruebas
await run_service_desk_workflow("¿Cuántos días de vacaciones tengo según la política?")
await run_service_desk_workflow("Quiero pedir mis vacaciones del 1 al 15 de agosto.")
await run_service_desk_workflow("No puedo conectarme a la VPN y tengo una reunión urgente con un cliente.")



🚀 Ejecutando workflow Service Desk con input:
"¿Cuántos días de vacaciones tengo según la política?"

📡 Evento: WorkflowStartedEvent(origin=WorkflowEventSource.FRAMEWORK, data=None)
📡 Evento: WorkflowStatusEvent(state=WorkflowRunState.IN_PROGRESS, data=None, origin=WorkflowEventSource.FRAMEWORK)
🧭 [TurnManager.start] Enviando AgentExecutorRequest al agente.
📡 Evento: ExecutorInvokedEvent(executor_id=turn_manager, data=None)
📡 Evento: ExecutorCompletedEvent(executor_id=turn_manager, data=None)
📡 Evento: ExecutorInvokedEvent(executor_id=service_desk_exec, data=None)
{
    "name": "message.send",
    "context": {
        "trace_id": "0x21dc3e3f8333a044987802f0c618c30c",
        "span_id": "0x137eed763b303dcb",
        "trace_state": "[]"
    },
    "kind": "SpanKind.PRODUCER",
    "parent_id": "0x84a4dbf77041aa97",
    "start_time": "2025-11-28T12:35:35.540510Z",
    "end_time": "2025-11-28T12:35:35.540691Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
     

{
    "name": "chat gpt-4o",
    "context": {
        "trace_id": "0x03fe66f99ab8789e4f31fa7be328ce35",
        "span_id": "0x2b65d2a1912c7e7f",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x1ba9a78adce9f375",
    "start_time": "2025-11-28T12:35:40.684246Z",
    "end_time": "2025-11-28T12:35:46.180660Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "gen_ai.input.messages": "[{\"role\": \"user\", \"parts\": [{\"type\": \"text\", \"content\": \"\\u00bfCu\\u00e1ntos d\\u00edas de vacaciones tengo seg\\u00fan la pol\\u00edtica?\"}]}, {\"role\": \"assistant\", \"parts\": [{\"type\": \"text\", \"content\": \"\\u00a1Hola! Esta consulta corresponde al \\u00e1rea de Recursos Humanos (RRHH). \\n\\nLa cantidad de d\\u00edas de vacaciones que tienes depende de la pol\\u00edtica interna de la empresa y, en algunos casos, de tu antig\\u00fcedad en la organizaci\\u00f3n. Te recomendar\\u00eda revisar el manual del empleado

In [ ]:


viz = WorkflowViz(workflow)

print("📜 Diagrama Mermaid:\n")
print(viz.to_mermaid())

print("\n📜 Diagrama DOT:\n")
print(viz.to_digraph())


📜 Diagrama Mermaid:

flowchart TD
  turn_manager["turn_manager (Start)"];
  service_desk_exec["service_desk_exec"];
  internal_turn_manager --> turn_manager;
  internal_service_desk_exec --> service_desk_exec;
  turn_manager --> service_desk_exec;
  service_desk_exec --> turn_manager;

📜 Diagrama DOT:

digraph Workflow {
  rankdir=TD;
  node [shape=box, style=filled, fillcolor=lightblue];
  edge [color=black, arrowhead=vee];

  "turn_manager" [fillcolor=lightgreen, label="turn_manager\n(Start)"];
  "service_desk_exec" [label="service_desk_exec"];
  "internal:turn_manager" -> "turn_manager";
  "internal:service_desk_exec" -> "service_desk_exec";
  "turn_manager" -> "service_desk_exec";
  "service_desk_exec" -> "turn_manager";
}


{
    "resource_metrics": [
        {
            "resource": {
                "attributes": {
                    "telemetry.sdk.language": "python",
                    "telemetry.sdk.name": "opentelemetry",
                    "telemetry.sdk.version": "1.38.0",
                    "service.name": "agent_framework"
                },
                "schema_url": ""
            },
            "scope_metrics": [
                {
                    "scope": {
                        "name": "agent_framework",
                        "version": "1.0.0b251120",
                        "schema_url": "",
                        "attributes": null
                    },
                    "metrics": [
                        {
                            "name": "gen_ai.client.token.usage",
                            "description": "Captures the token usage of chat clients",
                            "unit": "tokens",
                            "data": {
                            

In [ ]:
# Requiere graphviz instalado en el sistema para exportar a SVG y PNG
# https://graphviz.org/download/ 

# Exportar a SVG
svg_path = viz.export(format="svg")
print("✅ SVG generado en:", svg_path)

# Exportar a PNG
png_path = viz.export(format="png")
print("✅ PNG generado en:", png_path)


{
    "resource_metrics": [
        {
            "resource": {
                "attributes": {
                    "telemetry.sdk.language": "python",
                    "telemetry.sdk.name": "opentelemetry",
                    "telemetry.sdk.version": "1.38.0",
                    "service.name": "agent_framework"
                },
                "schema_url": ""
            },
            "scope_metrics": [
                {
                    "scope": {
                        "name": "agent_framework",
                        "version": "1.0.0b251120",
                        "schema_url": "",
                        "attributes": null
                    },
                    "metrics": [
                        {
                            "name": "gen_ai.client.token.usage",
                            "description": "Captures the token usage of chat clients",
                            "unit": "tokens",
                            "data": {
                            